### カテゴリ列のエンコーディング方法
カテゴリ列のエンコーディングといっても、  
ラベルエンコーディングとOneHotエンコーディング以外にも色々あるようなので、ためしてみる。  
元ネタ  
[kaggle:Category Encoders Examples](https://www.kaggle.com/discdiver/category-encoders-examples)

※カテゴリ列  
 └ 順序特徴量 … 数値ではないが順序があるもの。 例えば、サイズ(Sサイズ < Mサイズ < Lサイズ)  
 └ 名義特徴量 … 数値ではなく順序もないもの。 例えば、色(赤、青、黄色)

In [6]:
import numpy as np
import pandas as pd
import category_encoders  as ce
from sklearn.preprocessing import LabelEncoder
pd.options.display.float_format = '{:.2f}'.format 

In [178]:
df = pd.DataFrame({
        "COLOR" : ["Red", "Yellow", "Red", "Red", "Blue"],
        "SIZE" : ["S", "M", "L", "S", "S"],
        "WEIGHT" : [200, 100, 300, 300, 300],
        "HARDNESS" : ["hard", "soft", "medium", "hard", "soft"],
        "OUTCOME" : [1,2,0,0,0] })
df

,COLOR,SIZE,WEIGHT,HARDNESS,OUTCOME
0,Red,S,200,hard,1
1,Yellow,M,100,soft,2
2,Red,L,300,medium,0
3,Red,S,300,hard,0
4,Blue,S,300,soft,0


In [179]:
# 説明変数X と 目的変数Yに分ける
df_X = df[["COLOR", "SIZE", "WEIGHT","HARDNESS"]].copy()
df_Y = df[["OUTCOME"]].copy()

### ====================================
### LabelEncoder
sklearn.preprocessing.LabelEncoder()
[▼DOC](
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder
)  
> 0 から n_classes-1 の間の値を持つターゲットラベルをエンコードします。  
この変換器は、ターゲット値、すなわち y をエンコードするために使用されるべきであり、  
入力 X ではありません。  

・元の値を整数値に変換する。元々数値でも変換できる。    
・数値として扱えるようになるが、  
　名義特徴量(COLORとか)については数値になったことで順序ができてしまうのがNG。   
・数値のマッピング0,1,2,,,は、元の値の昇順の模様。  
・Nanがあると変換できずにエラーになる  
・Xも変換できるのだろうけど、そもそもYの変換を目的に作られているのかな？  
　たしかにデータフレーム丸ごと変換できない、Nanあるとダメってことからもそんな感じがする。  
　勉強に使ってたテキストとかはフツーにXに使ってたけども。  

In [180]:
print("エンコード前")
display(df_X)

print("エンコード後")
le = LabelEncoder()

# 1列ずつSeriesにしてエンコード
df_X_le = pd.DataFrame({
    "COLOR" : le.fit_transform(df_X["COLOR"]),
    "SIZE"  : le.fit_transform(df_X["SIZE"]),
    "WEIGHT"  : le.fit_transform(df_X["WEIGHT"]),
    "HARDNESS"  : le.fit_transform(df_X["HARDNESS"]),
})
display(df_X_le)

エンコード前


,COLOR,SIZE,WEIGHT,HARDNESS
0,Red,S,200,hard
1,Yellow,M,100,soft
2,Red,L,300,medium
3,Red,S,300,hard
4,Blue,S,300,soft


エンコード後


,COLOR,SIZE,WEIGHT,HARDNESS
0,1,2,1,0
1,2,1,0,2
2,1,0,2,1
3,1,2,2,0
4,0,2,2,2


In [181]:
# 数値のマッピングは、元の値の昇順ぽいので
# 自分で指定したいときは、接頭語つけてあげればいい
df_X_tmp = df_X.copy()
df_X_tmp["HARDNESS"] = df_X_tmp["HARDNESS"].mask(df_X_tmp["HARDNESS"]=="hard",   "2_hard")
df_X_tmp["HARDNESS"] = df_X_tmp["HARDNESS"].mask(df_X_tmp["HARDNESS"]=="medium", "1_medium")
df_X_tmp["HARDNESS"] = df_X_tmp["HARDNESS"].mask(df_X_tmp["HARDNESS"]=="soft",   "0_soft")

print("エンコード前")
display(df_X_tmp[["HARDNESS"]])

print("エンコード後")
le = LabelEncoder()

# 1列ずつSeriesにしてエンコード
df_X_le = pd.DataFrame({
    "HARDNESS"  : le.fit_transform(df_X_tmp["HARDNESS"]),
})
display(df_X_le)

エンコード前


,HARDNESS
0,2_hard
1,0_soft
2,1_medium
3,2_hard
4,0_soft


エンコード後


,HARDNESS
0,2
1,0
2,1
3,2
4,0


In [201]:
# Nanがあると変換できない
df_X_tmp = pd.DataFrame({
        "COLOR" : ["Red", "Yellow", "Red", np.nan, "Blue"],
        "HARDNESS" : ["hard", np.nan, "medium", "hard", "soft"] })

#print("エンコード前")
#display(df_X_tmp)

#print("エンコード後")
#le = LabelEncoder()

try:
    # 1列ずつSeriesにしてエンコード
    df_X_le = pd.DataFrame({
        "COLOR" : le.fit_transform(df_X_tmp["COLOR"]),
        "HARDNESS"  : le.fit_transform(df_X_tmp["HARDNESS"]),
    })
    display(df_X_le)

except TypeError as e:
    print("エラー:", e)

エラー: argument must be a string or number


### ====================================
### OrdinalEncoder (Ordinal=順序)
sklearn.preprocessing.OrdinalEncoder
[▼DOC](
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html
)  
> この変換器への入力は  
カテゴリ的（離散的）特徴量によって  
取られる値を表す**整数または文字列の配列**のようなものでなければなりません．  
特徴量は，**順序の整数**に変換されます．  
これにより，特徴量ごとに1列の整数（0からn_categories - 1）が得られます．

・元々数値の場合は変換されないのかな？  
・数値のマッピング1,2,,,は、もとの値の登場順に振っている模様。  
・データフレーム丸ごと変換できる。  
・nanがあっても変換できる。

In [152]:
print("エンコード前")
display(df_X)

print("エンコード後")
oe = ce.OrdinalEncoder()
df_X_oe = oe.fit_transform(df_X)
display(df_X_oe)

エンコード前


,COLOR,SIZE,WEIGHT,HARDNESS
0,Red,S,200,hard
1,Yellow,M,100,soft
2,Red,L,300,medium
3,Red,S,300,hard
4,Blue,S,300,soft
5,Blue,L,100,medium


エンコード後


,COLOR,SIZE,WEIGHT,HARDNESS
0,1,1,200,1
1,2,2,100,2
2,1,3,300,3
3,1,1,300,1
4,3,1,300,2
5,3,3,100,3


In [168]:
# 数値のマッピング1,2,,,は、もとの値の登場順に振っているっぽい。
# ので、自分で指定したい場合は、順序のリストを渡すといい。
print("エンコード前")
display(df_X[["HARDNESS"]])

# 都度、新しくインスタンスを作成して
# 自分が指定した順序を覚えさせて、変換する。
# 複数列まとめて指定する方法もあるんだろうけどわからなかった。。。
oe = ce.OrdinalEncoder()
order = [['soft'], ['medium'], ['hard']] # この順で、1,2,3,,を振る
oe.fit(order)
df_X_le = oe.transform(np.ravel(df_X[["HARDNESS"]]))

print("エンコード後")
df_X_le.columns = ["HARDNESS"]
display(df_X_le)

エンコード前


,HARDNESS
0,hard
1,soft
2,medium
3,hard
4,soft
5,medium


エンコード後


,HARDNESS
0,3
1,1
2,2
3,3
4,1
5,2


In [200]:
# Nanがあっても変換できる
df_X_tmp = pd.DataFrame({
        "COLOR" : ["Red", "Yellow", "Red", np.nan, "Blue"],
        "HARDNESS" : ["hard", np.nan, "medium", "hard", "soft"] })

#print("エンコード前")
display(df_X_tmp)

print("エンコード後")
oe = ce.OrdinalEncoder()
oe.fit_transform(df_X_tmp)

,COLOR,HARDNESS
0,Red,hard
1,Yellow,NaN
2,Red,medium
3,NaN,hard
4,Blue,soft


エンコード後


,COLOR,HARDNESS
0,1,1
1,2,2
2,1,3
3,3,1
4,4,4


### ====================================
### One-Hot Encoder (Ordinal:順序)
[▼DOC](
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html
)  
> この変換器への入力は  
カテゴリ的（離散的）特徴量によって  
取られる値を表す**整数または文字列の配列**のようなものでなければなりません．  
特徴量は，**順序の整数**に変換されます．  
これにより，特徴量ごとに1列の整数（0からn_categories - 1）が得られます．

ほうほうが２つあること、
Nanha？？

In [ ]:
oe2.transform(df_X)

In [141]:
pd.DataFrame(df["HARDNESS"], columns = ["HARDNESS":"0"])

SyntaxError: invalid syntax (<ipython-input-141-7a9b0d21cd47>, line 1)

In [137]:
df["HARDNESS"].valu

0      hard
1      soft
2    medium
3      hard
4      soft
5    medium
Name: HARDNESS, dtype: object

In [142]:
np.ravel(df["HARDNESS"])

array(['hard', 'soft', 'medium', 'hard', 'soft', 'medium'], dtype=object)